# Policy Impact Analysis
This notebook iterates through 6 different policy reforms and calculates their 10-year budgetary impacts.

In [ ]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

## Define Policy Reform Functions
Each function returns a Reform object for different policy options

In [ ]:
# Option 1: Tax Credit Expansion
def get_repeal_social_security_reform():
    # Placeholder for Tax Credit Expansion reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        }
        }, country_id="us")

# Policy 2: Infrastructure Investment
def get_flat_ss_tax_reform():
    # Placeholder for Infrastructure Investment reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2024-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2024-01-01.2100-12-31": 0
        }
        }, country_id="us")

# Policy 3: Healthcare Reform
def get_flat_ss_tax_with_senior_deduction_extension_reform():
    # Placeholder for Healthcare Reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        # Add your reform parameters here
    }, country_id="us")

# Policy 4: Child Tax Credit (with variable amounts)
def get_ss_credit_reform(amount):
    return Reform.from_dict({
        "gov.contrib.crfb.ss_credit.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.ss_credit.amount.SINGLE": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.JOINT": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.SEPARATE": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.irs.deductions.senior_deduction.amount": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
    }, country_id="us")

# Policy 5: Education Funding
def get_roth_style_swap_reform():
    # Placeholder for Education Funding reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2025-01-01.2100-12-31": True
        }
        }, country_id="us")

# Policy 6: Green Energy Incentives
def get_roth_style_swap_with_phase_in_reform():
    # Placeholder for Green Energy Incentives reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        # Add your reform parameters here
    }, country_id="us")

In [21]:
# Pre-compute baselines for all years to avoid redundant calculations
print("Pre-computing baselines for all years...")
baselines = {}
years = list(range(2025, 2035))

for year in years:
    print(f"  Computing baseline for {year}...")
    baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
    baseline_income = baseline.calculate("household_net_income", period=year)
    baselines[year] = baseline_income
    
print("Baseline computation complete!\n")

  Computing baseline for 2026...
  Computing baseline for 2027...
  Computing baseline for 2028...
  Computing baseline for 2029...
  Computing baseline for 2030...
  Computing baseline for 2031...
  Computing baseline for 2032...
  Computing baseline for 2033...
  Computing baseline for 2034...
Baseline computation complete!



## Pre-compute Baselines
Computing baselines once for each year to improve performance

In [22]:
def calculate_yearly_impact(reform, year, baseline_income):
    """
    Calculate the budgetary impact for a given reform and year.
    Uses pre-computed baseline to avoid redundant calculations.
    Returns impact in billions (negative = cost, positive = savings)
    """
    try:
        if reform is not None:
            reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
            reformed_income = reformed.calculate("household_net_income", period=year)
            difference_income = reformed_income - baseline_income
            impact_billions = difference_income.sum() / 1e9
        else:
            # Return placeholder value if reform not implemented
            impact_billions = 0.0
            
        return round(impact_billions, 1)
    except Exception as e:
        print(f"Error calculating impact for year {year}: {e}")
        return 0.0

def save_results_incrementally(results, filename='policy_impacts_checkpoint.csv'):
    """
    Save results incrementally to avoid data loss
    """
    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)
    return df

## Process All 6 Policy Reforms
Iterate through each policy and calculate 10-year impacts

In [ ]:
# Define years to analyze
years = list(range(2025, 2035))

# Initialize results list
all_results = []

# Policy configurations
policies = [
    {
        'id': 1,
        'name': 'Repeal Social Security benefits taxation',
        'reform_func': get_repeal_social_security_reform,
        'has_variants': False
    },
    {
        'id': 2,
        'name': 'Flat Social Security Tax',
        'reform_func': get_flat_ss_tax_reform,
        'has_variants': False
    },
    {
        'id': 3,
        'name': 'Flat Social Security Tax with Bonus Senior Deduction extension',
        'reform_func': get_flat_ss_tax_with_senior_deduction_extension_reform,
        'has_variants': False
    },
    {
        'id': 4,
        'name': 'Social Security Credit',
        'reform_func': get_ss_credit_reform,
        'has_variants': True,
        'variants': [300, 600, 900, 1200, 1500]
    },
    {
        'id': 5,
        'name': 'Roth Style Swap',
        'reform_func': get_roth_style_swap_reform,
        'has_variants': False
    },
    {
        'id': 6,
        'name': 'Roth Style Swap with Phase-In',
        'reform_func': get_roth_style_swap_with_phase_in_reform,
        'has_variants': False
    }
]

In [ ]:
# Process each policy with incremental saving
import os

# First check if baselines are computed
if 'baselines' not in locals():
    print("Baselines not found. Computing baselines first...")
    baselines = {}
    years = list(range(2025, 2035))
    
    for year in years:
        print(f"  Computing baseline for {year}...")
        baseline = Microsimulation()
        baseline_income = baseline.calculate("household_net_income", period=year)
        baselines[year] = baseline_income
    
    print("Baseline computation complete!\n")

# Check if checkpoint file exists to resume
checkpoint_file = 'policy_impacts_checkpoint.csv'
if os.path.exists(checkpoint_file):
    print(f"Found existing checkpoint file: {checkpoint_file}")
    existing_df = pd.read_csv(checkpoint_file)
    all_results = existing_df.to_dict('records')
    print(f"Loaded {len(all_results)} existing results")
    
    # Determine which policies/years have been completed
    completed_keys = set()
    for result in all_results:
        key = (result['policy_id'], result.get('credit_value'), result['year'])
        completed_keys.add(key)
else:
    all_results = []
    completed_keys = set()

save_counter = 0  # Counter to track when to save

for policy in policies:
    print(f"\nProcessing Policy {policy['id']}: {policy['name']}")
    
    if policy['has_variants']:
        # Handle policies with multiple variants (e.g., Child Tax Credit)
        for variant in policy['variants']:
            print(f"  Variant: ${variant}")
            
            # Check if we need to process this variant
            needs_processing = False
            for year in years:
                key = (policy['id'], variant, year)
                if key not in completed_keys:
                    needs_processing = True
                    break
            
            if not needs_processing:
                print(f"    Already completed, skipping...")
                continue
                
            reform = policy['reform_func'](variant)
            
            for year in years:
                key = (policy['id'], variant, year)
                if key in completed_keys:
                    print(f"    Year {year}: Already computed, skipping...")
                    continue
                    
                print(f"    Year {year}: Computing...")
                impact = calculate_yearly_impact(reform, year, baselines[year])
                
                all_results.append({
                    'policy_id': policy['id'],
                    'policy_name': policy['name'],
                    'credit_value': variant,
                    'year': year,
                    'impact_billions': impact
                })
                
                save_counter += 1
                # Save every 10 calculations
                if save_counter % 10 == 0:
                    save_results_incrementally(all_results, checkpoint_file)
                    print(f"      Checkpoint saved ({len(all_results)} total results)")
    else:
        # Handle standard policies
        # Check if we need to process this policy
        needs_processing = False
        for year in years:
            key = (policy['id'], None, year)
            if key not in completed_keys:
                needs_processing = True
                break
        
        if not needs_processing:
            print(f"  Already completed, skipping...")
            continue
            
        try:
            reform = policy['reform_func']()
        except:
            # Use None if reform function not implemented
            reform = None
            print(f"  Note: Using placeholder values (reform not yet implemented)")
        
        for year in years:
            key = (policy['id'], None, year)
            if key in completed_keys:
                print(f"  Year {year}: Already computed, skipping...")
                continue
                
            print(f"  Year {year}: Computing...")
            impact = calculate_yearly_impact(reform, year, baselines[year])
            
            all_results.append({
                'policy_id': policy['id'],
                'policy_name': policy['name'],
                'credit_value': None,
                'year': year,
                'impact_billions': impact
            })
            
            save_counter += 1
            # Save every 10 calculations
            if save_counter % 10 == 0:
                save_results_incrementally(all_results, checkpoint_file)
                print(f"    Checkpoint saved ({len(all_results)} total results)")

# Final save
save_results_incrementally(all_results, checkpoint_file)
print(f"\nProcessing complete! Total results: {len(all_results)}")
print(f"Checkpoint saved to: {checkpoint_file}")


Processing Policy 1: Tax Credit Expansion
  Year 2025: Computing...
  Year 2026: Computing...
  Year 2027: Computing...
  Year 2028: Computing...
  Year 2029: Computing...
  Year 2030: Computing...
  Year 2031: Computing...
  Year 2032: Computing...
  Year 2033: Computing...
  Year 2034: Computing...
    Checkpoint saved (10 total results)

Processing Policy 2: Flat Social Security Tax
  Year 2025: Computing...
  Year 2026: Computing...
  Year 2027: Computing...
  Year 2028: Computing...
  Year 2029: Computing...
  Year 2030: Computing...
  Year 2031: Computing...
  Year 2032: Computing...
  Year 2033: Computing...
  Year 2034: Computing...
    Checkpoint saved (20 total results)

Processing Policy 3: Flat Social Security Tax with Bonus Senior Deduction extension
  Year 2025: Computing...
  Year 2026: Computing...
  Year 2027: Computing...
  Year 2028: Computing...
  Year 2029: Computing...
  Year 2030: Computing...
  Year 2031: Computing...
  Year 2032: Computing...
  Year 2033: Comp

In [ ]:
# Create DataFrame from results
df_results = pd.DataFrame(all_results)

# Display summary statistics
print("\n=== Summary Statistics ===")
print(f"Total records: {len(df_results)}")
print(f"\nPolicies processed:")
for policy_id, group in df_results.groupby('policy_id'):
    policy_name = group['policy_name'].iloc[0]
    if group['credit_value'].notna().any():
        variants = group['credit_value'].dropna().unique()
        print(f"  {policy_id}. {policy_name} - {len(variants)} variants")
    else:
        print(f"  {policy_id}. {policy_name}")

# Display first few rows
print("\n=== First 10 rows ===")
display(df_results.head(10))

In [ ]:
# Save final results to CSV files
output_file = 'policy_impacts.csv'
df_results.to_csv(output_file, index=False)
print(f"Final results saved to {output_file}")

# Also save to the React app's public folder
react_output = 'policy-impact-dashboard/public/policy_impacts.csv'
df_results.to_csv(react_output, index=False)
print(f"Results also saved to {react_output}")

# Clean up checkpoint file if everything succeeded
if os.path.exists(checkpoint_file):
    os.remove(checkpoint_file)
    print(f"Checkpoint file removed: {checkpoint_file}")